In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [12:40<00:00, 19.26it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
357488,AM Remix,82,2021-08-07,"Nio Garcia, J Balvin, Bad Bunny",https://open.spotify.com/track/05bfbizlM5AX6Mf...,United States,top200,MOVE_UP,300233.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Nio+Garcia...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:05bfbizlM5AX6Mf1RRyMho,0.743,0.655,6.000,-5.015,0.000,0.159,0.114,0.000,0.076,0.785,172.031,audio_features,05bfbizlM5AX6Mf1RRyMho,https://api.spotify.com/v1/tracks/05bfbizlM5AX...,https://api.spotify.com/v1/audio-analysis/05bf...,261335.000,4.000,49,86,NaN,other
404460,INDUSTRY BABY (feat. Jack Harlow),16,2021-08-16,Lil Nas X,https://open.spotify.com/track/27NovPIUIRrOZoC...,United States,viral50,NEW_ENTRY,NaN,"27 Jul 2021, 03:53","05 Apr 2019, 14:19","Lil Nas X (born April 9, 1999) is an American ...","Lil Nas X (born April 9, 1999) is an American ...",Hip-Hop,rap,trap,country rap,hip hop,trap,Hip-Hop,pop rap,2020s,2021,spotify:track:27NovPIUIRrOZoCHxABJwK,0.736,0.704,3.000,-7.409,0.000,0.061,0.020,0.000,0.050,0.894,149.995,audio_features,27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000.000,4.000,1,81,hip-hop,hip-hop
428715,Life Goes On,41,2021-10-15,Oliver Tree,https://open.spotify.com/track/0eu4C55hL6x29mm...,United States,top200,MOVE_DOWN,482717.000,"17 Jan 2022, 17:37","08 Mar 2018, 14:50","Oliver Tree Nickell is an American vocalist, p...","Oliver Tree Nickell is an American vocalist, p...",electronic,electropop,indie pop,Hip-Hop,pop rock,Hip-Hop,electropop,indie pop,a subtle use of vocal harmony,extensive vamping,spotify:track:0eu4C55hL6x29mmeAjytzC,0.700,0.490,0.000,-5.187,1.000,0.076,0.186,0.000,0.117,0.569,79.982,audio_features,0eu4C55hL6x29mmeAjytzC,https://api.spotify.com/v1/tracks/0eu4C55hL6x2...,https://api.spotify.com/v1/audio-analysis/0eu4...,161803.000,4.000,9,146,electronic,electronic
393564,The Code (feat. Polo G),57,2020-11-09,King Von,https://open.spotify.com/track/5NOUjBikzECEJyZ...,United States,top200,MOVE_DOWN,354278.000,"07 May 2021, 15:35","25 Aug 2020, 18:52","Dayvon Daquan Bennett (August 9, 1994 – Novemb...","Dayvon Daquan Bennett (August 9, 1994 – Novemb...",drill,Hip-Hop,rap,chicago,Gangsta Rap,NaN,NaN,NaN,NaN,NaN,spotify:track:5NOUjBikzECEJyZMTxtErD,0.831,0.620,6.000,-8.397,0.000,0.338,0.058,0.000,0.119,0.568,133.170,audio_features,5NOUjBikzECEJyZMTxtErD,https://api.spotify.com/v1/tracks/5NOUjBikzECE...,https://api.spotify.com/v1/audio-analysis/5NOU...,202413.000,4.000,27,12,drill,other
322056,PIE,107,2017-07-01,"Future, Chris Brown",https://open.spotify.com/track/0NZUqq4iLQddokg...,United States,top200,MOVE_DOWN,237032.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Future,+Ch...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:669gYFBhezfYPXQP1hBMOX,0.872,0.699,1.000,-6.127,1.000,0.049,0.276,0.000,0.103,0.725,138.929,audio_features,669gYFBhezfYPXQP1hBMOX,https://api.spotify.com/v1/tracks/669gYFBhezfY...,https://api.spotify.com/v1/audio-analysis/669g...,196835.000,4.000,34,13,NaN,other


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.535
group        0.133
female       0.115
non-binary   0.026
Name: gender, dtype: float64

In [28]:
df[df['gender']=='group'].sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
61147,Feel It Still,16,2017-10-17,Portugal. The Man,https://open.spotify.com/track/6QgjcU0zLnzq5Or...,United States,top200,MOVE_DOWN,750767.000,"29 May 2018, 23:33","02 Feb 2006, 03:32",Portugal. The Man is an experimental indie roc...,Portugal. The Man is an experimental indie roc...,experimental,indie,seen live,indie rock,Progressive,indie,funk,pop,psychedelic pop,2010s,spotify:track:6QgjcU0zLnzq5OrUoSZ3OK,0.801,0.795,1.000,-5.115,0.000,0.050,0.042,0.000,0.072,0.754,79.028,audio_features,6QgjcU0zLnzq5OrUoSZ3OK,https://api.spotify.com/v1/tracks/6QgjcU0zLnzq...,https://api.spotify.com/v1/audio-analysis/6Qgj...,163253.000,4.000,13,394,experimental,other,group
204823,Girls Like You (feat. Cardi B),140,2019-03-02,Maroon 5,https://open.spotify.com/track/7fa9MBXhVfQ8P8D...,United States,top200,MOVE_UP,279311.000,"02 Jun 2018, 19:16","12 Feb 2006, 07:15",Maroon 5 is an American pop rock band that ori...,Maroon 5 is an American pop rock band that ori...,rock,pop,pop rock,alternative,alternative rock,pop,cardi b,2018,maroon 5,2010s,spotify:track:6V1bu6o1Yo5ZXnsCJU8Ovk,0.851,0.541,0.000,-6.825,1.000,0.051,0.568,0.000,0.130,0.448,124.959,audio_features,6V1bu6o1Yo5ZXnsCJU8Ovk,https://api.spotify.com/v1/tracks/6V1bu6o1Yo5Z...,https://api.spotify.com/v1/audio-analysis/6V1b...,235545.000,4.000,3,407,rock,rock,group
448129,BLACKOUT,9,2021-08-31,Turnstile,https://open.spotify.com/track/0bGImSqDB2ebdeo...,United States,viral50,MOVE_UP,NaN,NaN,"20 Dec 2010, 05:28","Turnstile is a hardcore band from Baltimore, w...","Turnstile is a hardcore band from Baltimore, w...",hardcore,hardcore punk,seen live,punk,nyhc,NaN,NaN,NaN,NaN,NaN,spotify:track:0bGImSqDB2ebdeoCidUC8o,0.301,0.807,6.000,-6.258,1.000,0.070,0.001,0.112,0.085,0.595,84.051,audio_features,0bGImSqDB2ebdeoCidUC8o,https://api.spotify.com/v1/tracks/0bGImSqDB2eb...,https://api.spotify.com/v1/audio-analysis/0bGI...,173280.000,4.000,9,13,hardcore,other,group
320885,No Vacancy,80,2017-05-20,OneRepublic,https://open.spotify.com/track/6XOYVSmNDjKUNMX...,United States,top200,MOVE_DOWN,294857.000,NaN,"14 May 2006, 03:52","OneRepublic is an American, self-proclaimed ""g...","OneRepublic is an American, self-proclaimed ""g...",rock,pop rock,alternative,alternative rock,indie,NaN,NaN,NaN,NaN,NaN,spotify:track:4QeoDcR16IHpmmgFGQDrCp,0.720,0.727,5.000,-3.882,1.000,0.049,0.056,0.000,0.138,0.481,99.994,audio_features,4QeoDcR16IHpmmgFGQDrCp,https://api.spotify.com/v1/tracks/4QeoDcR16IHp...,https://api.spotify.com/v1/audio-analysis/4Qeo...,223190.000,4.000,31,73,rock,rock,group
409841,Straightenin,60,2021-05-27,Migos,https://open.spotify.com/track/3X2r2CnHOJeV5Ye...,United States,top200,MOVE_DOWN,367354.000,NaN,"26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,NaN,NaN,NaN,NaN,NaN,spotify:track:3X2r2CnHOJeV5YekPTgBnK,0.847,0.629,9.000,-5.810,1.000,0.102,0.032,0.000,0.152,0.109,135.976,audio_features,3X2r2CnHOJeV5YekPTgBnK,https://api.spotify.com/v1/tracks/3X2r2CnHOJeV...,https://api.spotify.com/v1/audio-analysis/3X2r...,255532.000,4.000,23,28,hip-hop,hip-hop,group


In [30]:
df[df['artist'] == 'Ariana Grande'].to_csv('test.csv', sep = ';')

Age from artist

-- taking the data from the summary

In [ ]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d{1,2}, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

In [ ]:
df.sample()

# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid